<a href="https://colab.research.google.com/github/xinnazim/esaa/blob/main/DtRf1_self_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random

# os 관리를 위한 라이브러리
import os
import gc

In [30]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

In [31]:
seed_everything(42) # Seed 고정

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
### 전처리된 파일 불러오기

train = pd.read_parquet('/content/drive/MyDrive/air/train_pre.parquet')
test = pd.read_parquet('/content/drive/MyDrive/air/test_pre.parquet')
sample_submission = pd.read_csv('/content/drive/MyDrive/air/sample_submission.csv', index_col = 0)

In [ ]:
train.head(3)

,Month,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_Airport_ID,Destination_Airport_ID,Distance,Carrier_ID(DOT),Tail_Number,Delay,Holiday
1,8,460,624,13930,14869,1250.0,20304,310,NaN,0
4,1,540,619,14771,10157,250.0,20304,554,NaN,0
5,4,945,1011,11618,11278,199.0,20452,3430,1.0,0


In [ ]:
test.head(3)

,Month,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_Airport_ID,Destination_Airport_ID,Distance,Carrier_ID(DOT),Tail_Number,Holiday
0,12,716,779,12266,14683,191.0,19977,4382,0
1,9,900,1035,11618,10397,746.0,19790,1934,0
2,3,960,1155,13930,12953,733.0,19977,2144,0


# **3. 준지도학습**
- 라벨이 없는 데이터에 대한 라벨링 진행

## **3-0. 데이터 분리**

In [34]:
### 데이터 분리
# 레이블이 있는 데이터와 없는 데이터 분리

train_labeled = train.loc[train['Delay'].notnull(),:]
train_unlabeled = train.loc[train['Delay'].isnull(),:]

In [ ]:
train_labeled.shape[0]

172245

In [ ]:
train_unlabeled.shape[0]

502770

- 라벨링 된 데이터에 비해 라벨링 되지 **않은** 데이터의 수가 더 많다.

In [ ]:
### X,y 분리

# train
X_labeled = train_labeled.drop(['Delay'],axis = 1)
y_labeled = train_labeled['Delay']
X_unlabeled = train_unlabeled.drop(['Delay'],axis = 1)

print("X Labeled: ", X_labeled.shape)
print("y Labeled: ", y_labeled.shape)
print()
print("X Unlabeled: ", X_unlabeled.shape)

# test
X_test = test
print()
print("X Test: ", X_test.shape)

X Labeled:  (172245, 9)
y Labeled:  (172245,)

X Unlabeled:  (502770, 9)

X Test:  (1000000, 9)


## **3-1. 기본 모델 모델링(Base Model Modeling)**
- Base Model 선정 & 학습
  - 레이블이 **있는** 데이터만을 사용하여 모델을 학습시키면서 튜닝을 진행
- 기본 분류기인 Base Model(ex> LogisticRegression)의 하이퍼 파라미터 튜닝을 수행
  - GridSearchCV, RandomizedSearchCV, 또는 Bayesian optimization 등의 방법 활용
  - 주어진 데이터와 문제에 맞는 하이퍼파라미터 조합을 찾는 것이 중요

# 초기 모델링
# **랜덤포레스트**


## **3-2. Self Training**
- 초기에 레이블이 없는 데이터를 레이블링하여 일부 레이블이 있는 데이터를 생성한 후, 이 데이터를 활용해 모델을 학습시키는 방법
- 학습된 모델을 사용하여 레이블이 없는 데이터에 대한 예측을 수행하고, 예측 결과 중 신뢰할 수 있는 일부 데이터를 레이블링에 추가로 사용하여 새로운 학습용 데이터를 생성
> 해당 과정을 반복하여 모델을 업데이트하면서 점진적으로 더 많은 레이블을 확보해나가는 방식



> 라벨링 끝이 안나길래 함수 변경



In [40]:
### Self Training을 위한 함수

def run_self_training(self_training_model, X_labeled, y_labeled, X_unlabeled, early_stopping_rounds=5):
    iter = 0  # 반복 횟수 카운트를 위한 변수
    rounds_without_improvement = 0  # early stopping을 위한 카운트 변수

    while X_unlabeled.shape[0] > 0 and rounds_without_improvement < early_stopping_rounds:
        iter = iter + 1
        print("=== Iteration {} ===".format(iter))

        ## 1. 레이블이 있는(labeled) 데이터로 모델 학습
        self_training_model.fit(X_labeled, y_labeled)

        ## 2. 훈련된 모델로 레이블이 없는 데이터를 분류
        y_pred_unlabeled = self_training_model.predict(X_unlabeled)

        ## 3. 의사 레이블(pseudo label) 설정
        # 모델이 90% 이상 확신하는 데이터를 labeled 데이터에 추가
        confident_mask = self_training_model.predict_proba(X_unlabeled).max(axis=1) > 0.9
        X_confident = X_unlabeled[confident_mask]
        y_confident = y_pred_unlabeled[confident_mask]
        # 신뢰할 수 있는 데이터를 레이블링에 추가하여 새로운 학습용 데이터 구성
        # 레이블링 된 데이터의 수를 증가시키는 과정
        X_labeled = np.concatenate([X_labeled, X_confident], axis=0)
        y_labeled = np.concatenate([y_labeled, y_confident], axis=0)
        # 의사 레이블링 된 데이터들은 이제 레이블이 없는 데이터에서 제거
        X_unlabeled = X_unlabeled[~confident_mask]

        print("{}개 레이블링 완료".format(X_confident.shape[0]))

        ## Early Stopping 체크
        if X_confident.shape[0] <= 30:
            rounds_without_improvement += 1
            print("Early Stopping 카운트: {}/{}".format(rounds_without_improvement, early_stopping_rounds))
            print()
        else:
            rounds_without_improvement = 0
            print()

    print()
    print("레이블링 완료")

    # 라벨링 된 데이터 반환
    return X_labeled, y_labeled, X_unlabeled



In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.semi_supervised import SelfTrainingClassifier
base_model=RandomForestClassifier()

# Self Training 분류기 생성
self_training_model = SelfTrainingClassifier(base_model)

# 초기 모델링

In [37]:
#하이퍼파라미터 튜닝

from sklearn.model_selection import GridSearchCV

params={
    'n_estimators':[50],
    'max_depth':[3,5,7],
    'min_samples_leaf':[8,12,18],
    'min_samples_split':[8,16,20]
}

rf_clf=RandomForestClassifier(random_state=0, n_jobs=-1)
grid_cv=GridSearchCV(rf_clf, param_grid=params, cv=2, n_jobs=-1)
grid_cv.fit(X_labeled, y_labeled)

print('최적 하이퍼 파라미터:\n',grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))


최적 하이퍼 파라미터:
 {'max_depth': 3, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 50}
최고 예측 정확도: 0.8806


In [38]:
rf_base_model= RandomForestClassifier(max_depth=3, min_samples_leaf=8, min_samples_split= 8, n_estimators= 50)

In [ ]:
# Self-training을 통한 레이블링 수행

X_labeled, y_labeled, X_unlabeled = run_self_training(rf_base_model, X_labeled, y_labeled, X_unlabeled, early_stopping_rounds = 10)

In [42]:
print(X_labeled.shape[0])
print(X_unlabeled.shape[0]/(X_labeled.shape[0] + X_unlabeled.shape[0]))

392641
0.4183225557950564


In [43]:
### 최종 데이터 병합

X_train_final = X_labeled
y_train_final = y_labeled

# 최종 예측 모델링


In [44]:
rf_ssl = RandomForestClassifier()

#하이퍼파라미터 튜닝


params={
    'n_estimators':[50],
    'max_depth':[3,5,7],
    'min_samples_leaf':[8,12,18],
    'min_samples_split':[8,16,20]
}

# RandomForestClassifier 객체 생성 후 GridSearchCV 수행
grid_cv=GridSearchCV(rf_ssl, param_grid=params, cv=2, n_jobs=-1)
grid_cv.fit(X_train_final, y_train_final)

print('최적 하이퍼 파라미터:\n',grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))



최적 하이퍼 파라미터:
 {'max_depth': 3, 'min_samples_leaf': 18, 'min_samples_split': 20, 'n_estimators': 50}
최고 예측 정확도: 0.7808


In [46]:
final_classifier = RandomForestClassifier(max_depth=3, min_samples_leaf=18, min_samples_split= 20, n_estimators= 50)
final_classifier.fit(X_train_final, y_train_final)
y_test_pred = final_classifier.predict_proba(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [ ]:
sample_submission.head(3)

In [47]:
rf_submission = pd.DataFrame(data = y_test_pred, columns = sample_submission.columns,
                          index = sample_submission.index)

In [48]:
rf_submission.head(3)

,Not_Delayed,Delayed
ID,,
TEST_000000,0.038167,0.961833
TEST_000001,0.183341,0.816659
TEST_000002,0.221696,0.778304


In [49]:
rf_submission.to_csv('rf_submission.csv', index = True)